In [20]:
!pip install scikit-learn
!pip install bokeh

     |████████████████████████████████| 18.5 MB 8.8 MB/s            
     |████████████████████████████████| 3.1 MB 3.2 MB/s            
     |████████████████████████████████| 661 kB 3.1 MB/s            


In [28]:
TIJERAS = "tijeras"
PAPEL = "papel"
PIEDRA = "piedra"
options = [PIEDRA, TIJERAS, PAPEL]

In [29]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == PIEDRA and p2 == TIJERAS:
        result = 1
    elif p1 == PIEDRA and p2 == PAPEL:
        result = 2
    elif p1 == TIJERAS and p2 == PIEDRA:
        result = 2
    elif p1 == TIJERAS and p2 == PAPEL:
        result = 1
    elif p1 == PAPEL and p2 == PIEDRA:
        result = 1
    elif p1 == PAPEL and p2 == TIJERAS:
        result = 2
        
    return result

In [30]:
search_winner(PAPEL, TIJERAS)

2

In [31]:
test = [
    [PIEDRA, PIEDRA, 0],
    [PIEDRA, TIJERAS, 1],
    [PIEDRA, PAPEL, 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [32]:
from random import choice
def get_choice():
    return choice(options)


In [33]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: tijeras player2: papel Winner: 1 
player1: tijeras player2: tijeras Winner: 0 
player1: papel player2: tijeras Winner: 2 
player1: papel player2: piedra Winner: 1 
player1: piedra player2: papel Winner: 2 
player1: papel player2: papel Winner: 0 
player1: piedra player2: piedra Winner: 0 
player1: papel player2: papel Winner: 0 
player1: tijeras player2: papel Winner: 1 
player1: piedra player2: piedra Winner: 0 


In [58]:
def str_to_list(option):
    if option==PIEDRA:
        res = [1,0,0]
    elif option==TIJERAS:
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, [PIEDRA, TIJERAS, PAPEL]))
data_y = list(map(str_to_list, [PAPEL, PIEDRA, TIJERAS]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [59]:
from sklearn.neural_network import MLPClassifier

In [60]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [61]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(warm_start=True)


/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [62]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y       

In [63]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)


Player1: papel Player2 (modelo): [0.12079536 0.14157886 0.85433647] --> piedra
Comprobamos: p1 VS p2: 1
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [64]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break


Iter: 1 - score: {'win': 535, 'loose': 465} 53.5 %
Iter: 2 - score: {'win': 548, 'loose': 452} 54.8 %
Iter: 3 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 4 - score: {'win': 567, 'loose': 433} 56.7 %
Iter: 5 - score: {'win': 574, 'loose': 426} 57.4 %
Iter: 6 - score: {'win': 553, 'loose': 447} 55.3 %
Iter: 7 - score: {'win': 540, 'loose': 460} 54.0 %
Iter: 8 - score: {'win': 546, 'loose': 454} 54.6 %
Iter: 9 - score: {'win': 562, 'loose': 438} 56.2 %
Iter: 10 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 11 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 12 - score: {'win': 365, 'loose': 635} 36.5 %
Iter: 13 - score: {'win': 363, 'loose': 637} 36.3 %
Iter: 14 - score: {'win': 300, 'loose': 700} 30.0 %
Iter: 15 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 16 - score: {'win': 332, 'loose': 668} 33.2 %
Iter: 17 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 18 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 19 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 20 - score: {'w

Iter: 161 - score: {'win': 773, 'loose': 227} 77.3 %
Iter: 162 - score: {'win': 783, 'loose': 217} 78.3 %
Iter: 163 - score: {'win': 798, 'loose': 202} 79.8 %
Iter: 164 - score: {'win': 782, 'loose': 218} 78.2 %
Iter: 165 - score: {'win': 775, 'loose': 225} 77.5 %
Iter: 166 - score: {'win': 791, 'loose': 209} 79.1 %
Iter: 167 - score: {'win': 782, 'loose': 218} 78.2 %
Iter: 168 - score: {'win': 786, 'loose': 214} 78.6 %
Iter: 169 - score: {'win': 776, 'loose': 224} 77.6 %
Iter: 170 - score: {'win': 796, 'loose': 204} 79.6 %
Iter: 171 - score: {'win': 773, 'loose': 227} 77.3 %
Iter: 172 - score: {'win': 769, 'loose': 231} 76.9 %
Iter: 173 - score: {'win': 774, 'loose': 226} 77.4 %
Iter: 174 - score: {'win': 766, 'loose': 234} 76.6 %
Iter: 175 - score: {'win': 774, 'loose': 226} 77.4 %
Iter: 176 - score: {'win': 785, 'loose': 215} 78.5 %
Iter: 177 - score: {'win': 762, 'loose': 238} 76.2 %
Iter: 178 - score: {'win': 765, 'loose': 235} 76.5 %
Iter: 179 - score: {'win': 791, 'loose': 209} 

In [65]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()


Loading BokehJS ...

In [66]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y, legend='eficiencia', line_width=1)
show(p)


In [57]:
model.predict_proba([str_to_list(PIEDRA)])

array([[1.08200270e-04, 9.71400454e-05, 9.99795626e-01]])